In [15]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [16]:
try:
    __confing.execute_notebook('notebooks/0_data_extract.ipynb')
except:
    pass

In [17]:
import pandas as pd
import numpy as np
import random
import os
import optuna

from src import data_procces as dp
from src.feature_selection import fast_fill

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

c:\Users\Leo\OneDrive\Escritorio\Final Proyect\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Intoduction

In order to predict whether a person will be hospitalized, our main focus is to gather all the features related to their health status. To achieve this, we extensively search the documentation of the database [link] to identify and extract the relevant information. By analyzing these health-related features, we aim to build a predictive model that can effectively assess the likelihood of hospitalization for individuals. The database documentation serves as a valuable resource, providing comprehensive insights into the available variables and their significance in determining hospitalization outcomes. This thorough exploration allows us to capture the necessary data to develop a robust and accurate predictive model for hospitalization prediction.

# Functions

In [37]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    """
    Obtain the top 'n' feature importance values from a given model.
    
    Args:
        model (object): The trained model used for feature importance calculation.
        X (pd.DataFrame): The input dataframe containing the features.
        y (pd.Series): The target variable series.
        n (int): The number of top feature importances to return.
    
    Returns:
        list: A list of the top 'n' feature names with the highest importances.
    """
    
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Get the name of the target column from the Series y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return list(nueva_lista)



def drop_correlated_features(X, threshold=0.5):
    """Find correlated columns in a DataFrame and drop them 
    
    Arguments:
        df (DataFrame): Data to analize.
        threshold (float): Minimun correlation value considered to decide whether
        two columns are correlated or not.
    
    Rreturns:
        A list with non-correlated columns."""
    
    numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
    random.shuffle(numeric_features)  # Reorder the list randomly
    numeric_data = X[numeric_features].copy()
    corr = numeric_data.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape, dtype=bool), k=1))
    the_drop = [column for column in upper.columns if any(upper[column] >= threshold)]
    no_correlated_columns = list(numeric_data.drop(the_drop, axis=1).columns)
    
    return no_correlated_columns


def save_list(X, file_name):
    """
    Save a list of elements to a file.

    Args:
        X (list): The input list of elements.
        file_name (str): The name of the file to save the list.

    Returns:
        None

    """
    with open(file_name, 'w') as file:
        for element in X:
            file.write(str(element) + '\n')
    print(f"file {file_name}.")

# Feature Selection

To start, we utilize a dataframe that consists only of the selected columns. As per the project requirements, we aim to use 50 columns or fewer. We trained a model and obtained the columns that had the highest importance using a specific function.

By employing feature importance techniques, such as using the feature_importances_ attribute of a trained model, we are able to identify the most influential columns for predicting the hospitalization outcome. This function calculates the relative importance of each feature in contributing to the predictive power of the model.

In [18]:
# extracting CSV
df=pd.read_csv(r'data\custom\all_waves.csv')
target='pahosp1y'
selected = ['patoilt', 'pacncrsurg', 'padrinkcr', 'pamomage', 'paheight', 'pashophlp', 'pawthh', 'paarms', 'papaina', 'paosleep', 'paodngr', 'padadage', 'pawalkr', 'patoilethlp', 'parxhrtat', 'palgmusaa', 'padoctor1y', 'pameds', 'parxdiabo', 'parxdiab', 'paoangry', 'pahrtatte', 'paclims', 'pamealhlp', 'pahosp1y', 'papainfr', 'pabreast', 'paarthre', 'parafaany', 'parxarthr', 'paweight', 'pahipcomp', 'pagrossaa', 'padrinkbd', 'pamammog', 'parfaany', 'pamobilaa', 'pacncrradn', 'paprmem', 'pashop', 'paeat', 'pauppermoba', 'pacage', 'pastroklmt', 'pacholst', 'pabmi', 'parorgnz', 'paurina2y', 'pagender', 'papapsm', 'paprost', 'paopace', 'pabedhlp', 'parxhibp', 'paoplot', 'pamhip', 'padiabe', 'pafall', 'paglasses', 'pamoney', 'pahibpe', 'paoalchl', 'pasight', 'pawalkhlp', 'pahrtatlmt', 'padrinkb', 'pacncrothr', 'pawheeze', 'pasit', 'pastoop', 'pachair', 'paarthlmt', 'padress', 'paeathlp', 'pastroke', 'papainlv', 'parechrtatt', 'pabathehlp', 'pabreath_m', 'pafatigue', 'pabed', 'pacncrchem', 'pamoneyhlp', 'parxlung_m', 'palunglmt_m', 'pabath', 'palowermoba', 'pamedhlp', 'pahearaid', 'pameals', 'parxstrok', 'pafallinj', 'pahipe_m', 'parifaany', 'parjudg', 'paswell', 'pacncrmeds', 'pasmoken', 'padresshlp', 'pahigov', 'pahearing', 'palift', 'pasmokev', 'paagey', 'pacancre', 'pararcare', 'parfcaren']
print(f'number of columns {len(selected)}')

number of columns 107


In [19]:
# remove null from target variable
df = df.dropna(subset=[target]) 

# loading selected columns
df = df[selected].copy()

# removing rows with many nulls from majority class
nulls = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (nulls <= 0.5)]

# separating the target variable from the others
y = df[target]
X = df.drop(target,axis=1)

# filling the null values
X = fast_fill(X)

X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=42)

The initial step involved training and optimizing hyperparameters of a model using all the columns available. By utilizing techniques such as cross-validation and grid search, we fine-tuned the model's hyperparameters to achieve optimal performance. Once the model was trained, we evaluated its performance by calculating the AUC (Area Under the Curve) metric, which provides a measure of the model's ability to distinguish between positive and negative instances.

In [ ]:
def objective(trial):
    # Definir los hiperparámetros a optimizar
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 10, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }
    
    # Entrenar el modelo XGBClassifier con los hiperparámetros sugeridos por Optuna
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    
    # Calcular la predicción de probabilidad y el valor de AUC en el conjunto de validación
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    return auc_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_auc = study.best_value

After an exhaustive search for hyperparameters, we obtained the best possible hyperparameters and achieved the highest achievable AUC (Area Under the Curve) in 100 attempts. The resulting AUC value obtained was 91, showcasing the model's strong predictive performance. Notably, this exceptional AUC was achieved using a feature set comprising 107 carefully selected features.

## Feature importance

Afterwards, we proceed to extract the columns sorted by their feature importance scores, allowing us to conduct experiments by removing the columns with lower feature importance. This approach enables us to explore the impact of individual features on the model's predictive performance.
By removing the columns with relatively lower feature importance, we aim to refine the feature set and focus on the most influential predictors. This process ensures that the model's attention is primarily directed towards the key features that contribute significantly to hospitalization prediction.

In [ ]:
# splitting the data in train and test
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=42)
# applying oversampling to compensate for unbalanced data
X_train, y_train = dp.apply_smote(X_train, y_train)
# applying a scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

# instantiating and training the model
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

In [34]:
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC: {auc_score}')

AUC: 0.9164589087504955


In [40]:
#Obtaining which columns had the greatest impact to predict
_X=feature_importance(model, X, y, 107)
print(_X)

['paoalchl', 'pashop', 'padoctor1y', 'pagrossaa', 'parifaany', 'pahrtatte', 'parfaany', 'pararcare', 'pacncrsurg', 'pacncrchem', 'palift', 'pabathehlp', 'parxhrtat', 'pacncrothr', 'pacholst', 'parxstrok', 'pacncrradn', 'pamobilaa', 'parfcaren', 'pacncrmeds', 'pastroklmt', 'palunglmt_m', 'parafaany', 'pacancre', 'parxlung_m', 'pafallinj', 'pamedhlp', 'pahrtatlmt', 'parechrtatt', 'pabedhlp', 'pashophlp', 'pastroke', 'pahigov', 'paopace', 'pabath', 'pamealhlp', 'pahearaid', 'padresshlp', 'pawalkhlp', 'padrinkcr', 'paodngr', 'pauppermoba', 'paoangry', 'pasmoken', 'pamoney', 'paarthlmt', 'parxdiab', 'pahipcomp', 'pacage', 'paprost', 'patoilethlp', 'padrinkbd', 'padrinkb', 'pafatigue', 'paswell', 'parxdiabo', 'parxarthr', 'parxhibp', 'paeathlp', 'pamhip', 'pabreath_m', 'paclims', 'paosleep', 'pameals', 'padiabe', 'pameds', 'papaina', 'paurina2y', 'padress', 'parorgnz', 'pahipe_m', 'pabreast', 'pahearing', 'palowermoba', 'pamammog', 'pawheeze', 'paarms', 'paeat', 'papainlv', 'paweight', 'paag

In [45]:
X90 = _X[:90]
# splitting the data in train and test
X_train, X_test, y_train, y_test=dp.split_data(X[X90], y, test_size=0.2, random_state=42)
# applying oversampling to compensate for unbalanced data
X_train, y_train = dp.apply_smote(X_train, y_train)
# applying a scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

# instantiating and training the model
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC: {auc_score}')

AUC: 0.9137202766510688


In [46]:
X85 = _X[:85]

# splitting the data in train and test
X_train, X_test, y_train, y_test=dp.split_data(X[X85], y, test_size=0.2, random_state=42)
# applying oversampling to compensate for unbalanced data
X_train, y_train = dp.apply_smote(X_train, y_train)
# applying a scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

# instantiating and training the model
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC: {auc_score}')


AUC: 0.9142820749625744


In [47]:
X80 = _X[:80]
# splitting the data in train and test
X_train, X_test, y_train, y_test=dp.split_data(X[X80], y, test_size=0.2, random_state=42)
# applying oversampling to compensate for unbalanced data
X_train, y_train = dp.apply_smote(X_train, y_train)
# applying a scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

# instantiating and training the model
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC: {auc_score}')

AUC: 0.9099131504064771


After executing the experiments, we arrived at the conclusion that the most important columns were the first 85. We observed that adding more columns beyond this threshold did not significantly impact the AUC. However, reducing the number of columns below 85 resulted in a noticeable decline in performance.

These findings highlight the significance of the selected 85 columns in capturing the essential information needed for accurate hospitalization prediction. By including these influential features, the model achieved its optimal performance, as reflected by the highest AUC obtained.

The diminishing returns observed when adding more columns beyond the first 85 suggest that additional features do not contribute substantially to the predictive power of the model. On the other hand, reducing the number of columns below this threshold compromises the model's ability to capture crucial patterns and relationships, resulting in a decline in performance.

These conclusions underscore the importance of feature selection and emphasize the need to strike a balance between including relevant features and avoiding the inclusion of redundant or less informative ones. By leveraging the top 85 columns, we can ensure a robust and accurate predictive model for hospitalization outcomes while keeping the feature set manageable and interpretable.

## Removing corelated columns

Given the current situation where we have 85 features that need to be reduced to 50 or fewer, we face the challenge of dealing with multiple columns that reference the same concept but are expressed differently. In such cases, we have decided to eliminate columns based on their correlation with each other.

To identify and eliminate correlated columns, we employ various techniques such as computing the correlation matrix and applying a threshold to determine the maximum allowable correlation between columns. By setting a threshold value, we can remove one of the correlated columns while retaining the most informative one.

By eliminating correlated columns, we aim to reduce redundancy in the feature set and improve the model's performance. Removing highly correlated features helps avoid multicollinearity, which can negatively impact the model's interpretability and stability.

The process of reducing the feature set through correlation analysis allows us to prioritize the most diverse and non-redundant set of features, while still preserving the critical information necessary for accurate hospitalization prediction.

In [53]:
X_reduced=drop_correlated_features(X[X85], threshold=0.24)

X_train, X_test, y_train, y_test=dp.split_data(X[X_reduced], y, test_size=0.2, random_state=42)
X_train, y_train = dp.apply_smote(X_train, y_train)
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

model = XGBClassifier(**best_params,n_jobs=-1)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC: {auc_score}')

AUC: 0.9063271716459174


After realizing that it was possible to eliminate columns in this manner, bearing in mind that the dropped columns may vary depending on the order in which they are considered for removal, we decided to implement a heuristic approach to further refine the feature selection process. This heuristic aims to identify and remove columns with similar meanings or predictive power.

The heuristic considers the semantic similarity or predictive strength of the columns and utilizes domain knowledge or statistical techniques to determine their similarity. For instance, we can use techniques such as clustering, natural language processing (NLP), or statistical measures to identify groups of columns that share similar patterns or represent similar concepts.

By analyzing the groups of similar columns, we can make informed decisions about which columns to drop, ensuring that we retain the most informative and diverse set of features. This process helps us avoid redundancies and capture the essential information necessary for accurate hospitalization prediction.

Implementing this heuristic provides a more intelligent and context-aware approach to feature selection. By considering the predictive power and semantic relationships between columns, we can create a more refined feature set that optimizes the model's performance while maintaining a compact and interpretable set of features.

It is important to note that the specific implementation of the heuristic can vary depending on the dataset and problem domain. Careful consideration of the domain knowledge and data characteristics is crucial to ensure the effectiveness of the heuristic in identifying and removing columns with similar meanings.

In [63]:
# Define the folder path
folder_path = 'FILES/auto'

# Check if the folders already exist
if os.path.exists(folder_path):
    print("Folders already exist. Skipping folder creation.")

else:
    # Create the folders
    os.makedirs(folder_path)
    
    # Check if the folders are created
    if os.path.exists(folder_path):
        print("Folders created successfully.")
    else:
        print("Failed to create folders.")

Folders already exist. Skipping folder creation.


### File creation

To perform feature selection and save the selected columns based on their predictive power, we followed a multi-step process. Firstly, we generated different random states of the data to ensure variability in our analysis. This was achieved by employing techniques like k-fold cross-validation or train-test splits with different random seeds. These random states represent different partitions of the dataset, enabling us to evaluate the stability and robustness of our feature selection process.

Next, we shuffled the columns within each random state. By randomly reordering the columns, we aimed to eliminate any potential bias or ordering effects that might exist in the original dataset. This step ensured that our analysis focused solely on the relevance and predictive power of the features, independent of their initial order.

With the shuffled variables in place, we proceeded to train our model on each random state and evaluate its performance using the area under the curve (AUC) metric. Specifically, we aimed for an AUC threshold of 90 or higher as an indicator of strong predictive capability. If a particular model achieved an AUC of 90 or higher, we considered it to have excellent performance and proceeded to the next step.

When a model reached or surpassed the desired AUC threshold, we stored the names of all the columns used in a text file. This file served as a record of the selected features that demonstrated significant predictive power for hospitalization outcomes. We ensured to provide a specific path for saving this file, making it easily accessible for future reference, analysis, or model retraining.

By following this systematic approach, we leveraged different random states, shuffled the columns, and assessed model performance to identify the most influential features. This process allowed us to select and save the columns that exhibited a high degree of predictive power, ensuring that our subsequent modeling efforts are based on the most informative and impactful set of features.

It is important to note that this process assumes prior model training and focuses specifically on feature selection based on predictive performance. Additionally, the AUC threshold can be adjusted based on the specific requirements and objectives of the project.

In [ ]:
for i in range(3):
    while True:
        state = random.randint(0, 100)
        folder_path = f"FILES/auto/{state}"

        if os.path.exists(folder_path):
            print(f"The folder '{folder_path}' already exists. Generating another random number.")
        else:
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created successfully.")
            break

    for i in range(10):
        while True:
            random_number = random.uniform(0.21, 0.24)
            __X=drop_correlated_features(X, threshold=random_number)
            
            X_train, X_test, y_train, y_test=dp.split_data(X[__X], y, test_size=0.2, random_state=state)
            X_train, y_train = dp.apply_smote(X_train, y_train)
            X_train, X_test = dp.apply_standard_scaler(X_train, X_test)
            
            if len(__X) < 39:
                model = XGBClassifier(**params,n_jobs=-1)
                model.fit(X_train, y_train)
                
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                auc = roc_auc_score(y_test, y_pred_proba)
                
                if auc >= 0.9:
                    save_list(__X, f'FILES/auto/{state}/F{len(__X)}_AUC_{round(auc,2)}_Index_{i}.txt')
                    break

Considering that each random state takes approximately 1 hour to generate and evaluate 10 different text files, it is crucial to take into account the potential presence of data leakage, which might further increase the processing time. Therefore, it is advisable to limit the number of iterations in the cell to ensure a reasonable runtime. In our case, we have experimented with 10 different random states, which provides a good balance between thorough exploration and computational efficiency.

### TF-IDF

After generating all the files containing columns from successful models, we decided to apply the TF-IDF (Term Frequency-Inverse Document Frequency) technique. TF-IDF is a method used to quantify the importance of a word within a text by indicating how frequently it appears in the text and how rare it is across multiple texts.

By applying TF-IDF to our dataset, we obtained a ranking of feature importance. This ranking allowed us to discriminate and identify the most significant variables to be utilized in our final model. The TF-IDF scores provided a measure of the relative importance of each feature based on its occurrence and rarity within the dataset.

Initially, we create an empty list called corpus to store the contents of the TXT files. We traverse through the specified folder path using the os.walk() function, searching for TXT files. For each file found, we read its content and append it to the corpus list.


In [64]:
# List to store the content of TXT files
corpus = []
# Iterate through all folders and TXT files
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                content = f.read()
                corpus.append(content)

To prepare the text data for analysis, we tokenize the corpus using the word_tokenize() function, which breaks down the text into individual words or tokens. The tokenized corpus is stored in the tokenized_corpus list.

In [65]:
# Tokenization of words
tokenized_corpus = [word_tokenize(doc) for doc in corpus]

Then we apply the TF-IDF (Term Frequency-Inverse Document Frequency) technique using the TfidfVectorizer from the sklearn.feature_extraction.text module. This allows us to calculate the TF-IDF values for each word in the corpus.

In [67]:
# Applying TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(doc) for doc in tokenized_corpus])

# Obtaining unique terms (words) in the corpus
feature_names = tfidf_vectorizer.get_feature_names_out()

To create a list of words ordered by their TF-IDF values, we initialize an empty list called words_tfidf and a set called word_set to track the words that have already been added. We iterate through the tokenized corpus and feature names, checking if the TF-IDF value is greater than zero and if the term is not already in the word_set. If these conditions are met, we add the word and its TF-IDF value to the words_tfidf list and update the word_set.


In [68]:
# Creating a list of words ordered by TF-IDF without repeating words
words_tfidf = []
word_set = set()
for i, doc in enumerate(tokenized_corpus):
    for j, term in enumerate(feature_names):
        tfidf_value = tfidf_matrix[i, j]
        if tfidf_value > 0 and term not in word_set:
            words_tfidf.append((term, tfidf_value))
            word_set.add(term)

# Sorting the list by TF-IDF value
words_tfidf.sort(key=lambda x: x[1], reverse=True)

# Saving the sorted list of words to a text file
with open("FILES/tfidf.txt", "w") as f:
    for word, tfidf in words_tfidf:
        f.write(f"{word}: {tfidf}\n")

Finally, we sort the words_tfidf list in descending order based on the TF-IDF values. We save the sorted list of words, along with their corresponding TF-IDF values, to a text file named "tfidf.txt" located in the "FILES" directory using the open() function in write mode.

Once the file was generated along with the documentation, we proceeded to purge columns from the dataset. The criteria for column purging were based on identifying columns that had high repetition and could be transformed into questions for a questionnaire. These columns were considered redundant and unnecessary for our analysis.

Additionally, we decided not to utilize certain columns that contained highly predictive information but required input from a healthcare professional. These columns likely contained sensitive or specialized data that could not be easily obtained or filled in by non-experts.

By removing these columns, we aimed to streamline the dataset and focus on variables that were more accessible and applicable to our analysis. This purging process helped us eliminate redundant information and concentrate on features that were easier to collect or obtain responses for.

It's important to note that the specific columns chosen for purging may vary depending on the nature of the dataset, the goals of the analysis, and the availability of data. The criteria for column selection should align with the objectives of the project and the practical feasibility of obtaining and utilizing the data effectively.